In [ ]:
"""
Jordan Stobbs
Phishing AI test area

An area to test various optimised and unoptimised AI models for phishing detection

Version 1.4
"""

In [5]:
import sklearn
from html.parser import HTMLParser
import requests
from sklearn.model_selection import train_test_split
import sqlite3
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import tldextract
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.preprocessing import normalize
import re
from bs4 import BeautifulSoup as bs
from sklearn.preprocessing import StandardScaler, Binarizer
from selenium import webdriver
from keras.models import Sequential, load_model
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from tpot import TPOTClassifier
import numpy as np
from statistics import mean
import pickle

import pyswarms as pso
	
from mealpy.swarm_based.MFO import BaseMFO
#used to define how layers are created

from keras.layers import Conv2D, MaxPooling2D
#used for convolutional layer (using a 2D array)
from keras.layers.normalization import BatchNormalization
#used to smooth activation
from keras.optimizers import SGD, Adam, Adadelta, RMSprop

from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.losses import binary_crossentropy

Using TensorFlow backend.
/home/jordan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jordan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jordan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jordan/.local/lib/python3.7/site-packages/tensorflow/python

In [6]:
DATABASE_DIRECTORY = "data2.db"
URLSCAN_DOM = "https://urlscan.io/dom/"


In [7]:
#Checks if a certain tld is used
def tldAdd(tldCheck,tld):
    if(tldCheck.startswith(tld)):
        return 1
    else:
        return 0
        

In [8]:
#Adds a feature to the dataframe with a given name.
def addFeatureToDataframe(arrayAdd,name,df):
    assert len(df) == len(arrayAdd)
    df[name] = arrayAdd
    
    

In [9]:
con = sqlite3.connect(DATABASE_DIRECTORY)
cur = con.cursor()
cur.execute("SELECT url, urlClassification FROM Url")
results = cur.fetchall()
df = pd.DataFrame(data = results, columns=["url","classification"])
dashArray = []
domainLengthArray = []
underscoreArray = []
atArray = []
slashArray = []
equalsArray = []
httpArray = []
httpsArray = [] 
wwwArray = []
dotArray = []
digitArray = []
phpArray = []
htmlArray = []
cssArray = []
logArray = []
payArray = []
webArray = []
cmdArray = []
govArray = []
accountArray = []
dispatchArray = []
freeArray = []
eduArray = []
netArray = []
ruArray = []
cnArray = []
orgArray = []
jpArray = []
comBrArray = []
deArray = []
irArray = []
tvArray = []
coJpArray = []
grArray = []
plArray = []
esArray = []
infoArray = []
appArray = []
comAuArray = []
mxArray = []
comTrArray = []
# aspArray = [] Not used due to other words containing "asp" e.g. aspire
coukArray = []
ipContained = []
for result in results:
    url = result[0]
    ipInUrl = re.findall(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b",url)
    if(len(ipInUrl) == 0):
        ipContained.append(0)
    else:
        ipContained.append(1)
        
    if(url.startswith("http://")):
        urlNoHttpWww = url[7:]
        httpsArray.append(0)
    elif(url.startswith("https://")):
        urlNoHttpWww = url[8:]
        httpsArray.append(1)
    else:
        httpsArray.append(0)
        urlNoHttpWww = url
    if(urlNoHttpWww.startswith("www.")):
        urlNoHttpWww = urlNoHttpWww[3:]
    dashArray.append(url.count("-"))
    underscoreArray.append(url.count("_"))
    atArray.append(url.count("@"))
    slashArray.append(urlNoHttpWww.count("/"))
    equalsArray.append(url.count("="))
    if "http" in urlNoHttpWww:
        httpArray.append(True)
    else:
        httpArray.append(False)
    if "www" in urlNoHttpWww:
        wwwArray.append(True)
    else:
        wwwArray.append(False)
    brokenUrl = tldextract.extract(url)
    domain = brokenUrl.domain +"." + brokenUrl.suffix
    domainLengthArray.append(len(brokenUrl.domain))
    dotArray.append(domain.count("."))
    digitArray.append(sum(letter.isdigit() for letter in domain))
    
    if "html" in domain:
        htmlArray.append(1)
    else:
        htmlArray.append(0)
    if "css" in domain:
        cssArray.append(1)
    else:
        cssArray.append(0)
    if "php" in domain:
        phpArray.append(1)
    else:
        phpArray.append(0)
    
    if "log" in domain:
        logArray.append(1)
    else:
        logArray.append(0)
    if "pay" in domain:
        payArray.append(1)
    else:
        payArray.append(0)
    if "web" in domain:
        webArray.append(1)
    else:
        webArray.append(0)
    if "cmd" in domain:
        cmdArray.append(1)
    else:
        cmdArray.append(0)
    if "account" in domain:
        accountArray.append(1)
    else:
        accountArray.append(0)
    if "dispatch" in domain:
        dispatchArray.append(1)
    else:
        dispatchArray.append(0)  
    if "free" in domain:
        freeArray.append(1)
    else:
        freeArray.append(0)  
    
    
    tld = brokenUrl.suffix
    
    coukArray.append(tldAdd(tld,"co.uk"))
    govArray.append(tldAdd(tld,"gov"))
    eduArray.append(tldAdd(tld,"edu"))
    netArray.append(tldAdd(tld,"net"))
    ruArray.append(tldAdd(tld,"ru"))
    cnArray.append(tldAdd(tld,"cn"))
    orgArray.append(tldAdd(tld,"org"))
    jpArray.append(tldAdd(tld,"jp"))
    comBrArray.append(tldAdd(tld,"com.br"))
    deArray.append(tldAdd(tld,"de"))
    irArray.append(tldAdd(tld,"ir"))
    tvArray.append(tldAdd(tld,"tv"))
    coJpArray.append(tldAdd(tld,"co.jp"))
    grArray.append(tldAdd(tld,"gr"))
    plArray.append(tldAdd(tld,"pl"))
    esArray.append(tldAdd(tld,"es"))
    infoArray.append(tldAdd(tld,"info"))
    appArray.append(tldAdd(tld,"app"))
    comAuArray.append(tldAdd(tld,"com.au"))
    mxArray.append(tldAdd(tld,"mx"))
    comTrArray.append(tldAdd(tld,"com.tr"))



addFeatureToDataframe(dashArray,"dash",df)
addFeatureToDataframe(domainLengthArray,"domainLength",df)
addFeatureToDataframe(underscoreArray,"underscore",df)
addFeatureToDataframe(atArray,"at",df)
addFeatureToDataframe(slashArray,"slash",df)
addFeatureToDataframe(equalsArray,"equals",df)
addFeatureToDataframe(httpArray,"http",df)
addFeatureToDataframe(wwwArray,"www",df)
addFeatureToDataframe(dotArray,"dot",df)
addFeatureToDataframe(digitArray,"digit",df)
addFeatureToDataframe(coukArray,"coukTld",df)
addFeatureToDataframe(govArray,"govTld",df)
addFeatureToDataframe(eduArray,"eduTld",df)
addFeatureToDataframe(netArray,"netTld",df)
addFeatureToDataframe(ruArray,"ruTld",df)
# addFeatureToDataframe(cssArray,"containsCss",df)
# addFeatureToDataframe(phpArray,"containsPhp",df)
# addFeatureToDataframe(htmlArray,"containsHtml",df)
addFeatureToDataframe(logArray,"containsLog",df)
addFeatureToDataframe(payArray,"containsPay",df)
addFeatureToDataframe(webArray,"containsWeb",df)
addFeatureToDataframe(cmdArray,"containsCmd",df)
addFeatureToDataframe(accountArray,"containsAccount",df)
addFeatureToDataframe(dispatchArray,"containsDispatch",df)
# addFeatureToDataframe(freeArray,"containsFree",df)
addFeatureToDataframe(httpsArray,"containsHttps",df)
addFeatureToDataframe(orgArray,"orgTld",df)
addFeatureToDataframe(jpArray,"jpTld",df)
addFeatureToDataframe(comBrArray,"comBrTld",df)
addFeatureToDataframe(deArray,"deTld",df)
addFeatureToDataframe(cnArray,"cnTld",df)
addFeatureToDataframe(irArray,"irTld",df)
addFeatureToDataframe(tvArray,"tvTld",df)
addFeatureToDataframe(coJpArray,"coJpTld",df)
addFeatureToDataframe(grArray,"grTld",df)
addFeatureToDataframe(plArray,"plTld",df)
addFeatureToDataframe(esArray,"esTld",df)
addFeatureToDataframe(infoArray,"infoTld",df)
addFeatureToDataframe(appArray,"appTld",df)
addFeatureToDataframe(comAuArray,"comAuTld",df)
addFeatureToDataframe(mxArray,"mxTld",df)
addFeatureToDataframe(comTrArray,"comTrTld",df)
addFeatureToDataframe(ipContained,"ipInUrl",df)

In [10]:
con = sqlite3.connect(DATABASE_DIRECTORY)
cur = con.cursor()
cur.execute("SELECT url, urlIp, urlHash, urlAsn, urlCountry, certificateIssuer,\
             certificateValidFrom, certificateValidTo,DomNowhere, DomExternal, \
             urlGoogleSafeBrowsing, urlGoogleFrontPage\
             FROM Url LEFT JOIN certificate on url.urlId = certificate.urlId\
                      LEFT JOIN DomFeatures on url.urlId = DomFeatures.urlId")
results = cur.fetchall()

certificateIssuerArray = []
certificateDurationArray = []
urlIpArray = []
urlHashArray = []
urlCountryArray = []
urlAsnArray = []
urlFrontPage = []
urlSafeBrowsing = []
domExternalArray = []
domNowhereArray = []
tempUrl = "-1"
for row in results:
    url = row[0]
    if(url == tempUrl):
        continue
    else:
        tempUrl = url
    urlIp = row[1]
    urlHash = row[2]
    urlAsn = row[3]
    urlCountry = row[4]
    certificateIssuer = row[5]
    if(certificateIssuer == None or certificateIssuer == "None"):
        certificateIssuer = "None"
        certificateDuration = -1
    else:    
        certificateIssuer = row[5]
        certificateDuration = row[7]-row[6]
    domNowhere = row[8]
    domExternal = row[9]
    safeBrowsing = row[10]
    frontPage = row[11]
    urlIpArray.append(urlIp)
    urlHashArray.append(urlHash)
    urlAsnArray.append(urlAsn)
    urlCountryArray.append(urlCountry)
    certificateDurationArray.append(certificateDuration)
    certificateIssuerArray.append(certificateIssuer)
    domNowhereArray.append(domNowhere)
    domExternalArray.append(domExternal)
    urlFrontPage.append(frontPage)
    urlSafeBrowsing.append(safeBrowsing)
urlIpArray = pd.factorize(urlIpArray)[0]
urlAsnArray = pd.factorize(urlAsnArray)[0]
certificateIssuerArray = pd.factorize(certificateIssuerArray)[0]
urlCountryArray = pd.factorize(urlCountryArray)[0]

# addFeatureToDataframe(urlIpArray,"ip",df)
# addFeatureToDataframe(urlHashArray,"hash",df)
addFeatureToDataframe(domNowhereArray,"linksToNowhere",df)
addFeatureToDataframe(domExternalArray,"linksToExternal",df)
addFeatureToDataframe(urlCountryArray,"urlCountry",df)
addFeatureToDataframe(urlAsnArray,"asn",df)
addFeatureToDataframe(certificateDurationArray,"certificateDuration",df)
addFeatureToDataframe(certificateIssuerArray,"certificateIssuer",df)
addFeatureToDataframe(urlFrontPage,"googleFrontPage",df)
addFeatureToDataframe(urlSafeBrowsing,"googleSafeBrowsing",df)


In [11]:
con = sqlite3.connect(DATABASE_DIRECTORY)
cur = con.cursor()
cur.execute("SELECT COUNT(requestId)\
             FROM Url LEFT JOIN urlRequest on url.urlId = urlRequest.urlId\
             GROUP BY url")
resultsCountsAll = cur.fetchall()

#This command gets all dead requests and counts them for each URL, including 0s
cur.execute("SELECT url, 0\
             FROM URL\
             WHERE url not in (\
                 SELECT url\
                 FROM Url LEFT JOIN urlRequest on url.urlId = urlRequest.urlId\
                      LEFT JOIN request on request.requestId = urlRequest.requestId\
                 WHERE requestStatus like '4%' or requestStatus like '5%')\
             UNION\
             SELECT url, COUNT(request.requestId)\
             FROM Url LEFT JOIN urlRequest on url.urlId = urlRequest.urlId\
                      LEFT JOIN request on request.requestId = urlRequest.requestId\
             WHERE requestStatus like '4%' or requestStatus like '5%'\
             GROUP BY url")
resultsCountDead = cur.fetchall()

cur.execute("SELECT url.urlId, url, requestUrl, requestStatus\
             FROM Url LEFT JOIN urlRequest on url.urlId = urlRequest.urlId\
                      LEFT JOIN request on request.requestId = urlRequest.requestId\
             GROUP BY url.urlId")
results = cur.fetchall()

sortedResults = []
tempId = -1
tempArray = []
first = True
for result in results:
    currentId = result[0]
    if currentId != tempId and not first:
        sortedResults.append([result[1],tempArray])
        tempArray = []
    else:
        if first:
            first = False
    tempId = result[0]
    tempArray.append([result[2],result[3]])
for result in sortedResults:
    print(result)

['https://cpakaren.com/bin-cgi/second-wave/GetMyPayment.html?IRSStimulusOnline&bn=3a87f6b7c2088874&burlid=d001a6ea0b9cbe16', [[None, None]]]
['http://www.s5s.uno/?iR1087', [[None, None]]]
['https://ww2.yuriybobak.com/', [[None, None]]]
['http://rakuten.co.jp.gfrhgfdxvfdegfcasdfesdghtrsd.xyz', [[None, None]]]
['https://mise-a-jour-orange7.webnode.com.co/', [[None, None]]]
['https://jchfhygcnrfjntrvhretj.weebly.com/', [[None, None]]]
['https://attmmaileailbox.weebly.com/', [[None, None]]]
['https://career-att.weebly.com/', [[None, None]]]
['https://melhores-destaques-de-precos-baixos.com/produtos/783547416/?conjunto-de-panela-antiaderente-turin-10-peças-vermelha-tramontina&skullid=254858284', [[None, None]]]
['https://r-amazon.top/enavi/', [[None, None]]]
['https://grow.ucidity.com/HGD/cmd-login=775e967f814116889dfd1e9c545561b7/index.php?email=abuse@ionos.com&loginpage=mail.ionos.com/&reff=YjRhYjY4NWYxM2Y3NTc0ODE0MWFhNWRmMDdkMTFhYjA', [[None, None]]]
['https://cignesecampus.com/work/asse

['https://nextwebinar.org/wp-content/email/email/web/?e', [[None, None]]]
['https://microsoft-outlook-authenticate.ts.r.appspot.com/', [[None, None]]]
['https://www.afinephotographer.com/redrction-you-now-please-wait/wait.php', [[None, None]]]
['http://learnwithportals.com/id/soveleq', [[None, None]]]
['https://learnwithportals.com/id/soveleq', [[None, None]]]
['http://office-outlook365-account.ts.r.appspot.com/', [[None, None]]]
['https://www.projonmowave.com/wp-admin/includes/HJK/VN/wells/wells.htm', [[None, None]]]
['http://office365outlookmailboxuser.wn.r.appspot.com/', [[None, None]]]
['https://cbsujavrh.ts.r.appspot.com/login.microsoftonline.com/common/oauth2', [[None, None]]]
['https://user4739879320.du.r.appspot.com/login.microsoftonline.com/common/oauth2', [[None, None]]]
['https://learnwithportals.com/id/zasada123', [[None, None]]]
['http://quinaroja.com/swift/outlook.com/common', [[None, None]]]
['https://office-outlook365-account.ts.r.appspot.com/login.microsoftonline.com/c

['https://services.runescape.com-as.ru/m=weblogin/loginform868,489,3,80175310,1', [[None, None]]]
['https://secure.runescape.com-as.ru/m=weblogin/loginform110,621,188,23897797,1', [[None, None]]]
['http://www.51jianli.cn/images/?us.battle.net/login/en?ref=xyvrmzdus.battle', [[None, None]]]
['http://www.51jianli.cn/images?us.battle.net/login/en?ref=xyvrmzdus.battle', [[None, None]]]
['http://www.51jianli.cn/images/?http:/us.battle.net/login/en?ref=http:/xyvrm', [[None, None]]]
['http://www.51jianli.cn/images?http:/us.battle.net/login/en?ref=http:/xyvrm', [[None, None]]]
['https://services.runescape.com-as.ru/m=weblogin/loginform990,210,484,47413841,1', [[None, None]]]
['https://secure.runescape.com-as.ru/m=weblogin/loginform813,191,909,77570729,1', [[None, None]]]
['https://services.runescape.com-as.ru/m=weblogin/loginform883,113,576,59935105,1', [[None, None]]]
['https://secure.runescape.com-as.ru/m=weblogin/loginform489,935,166,54991896,1', [[None, None]]]
['https://services.runescape

['https://www.sosyalhile.com/member', [[None, None]]]
['http://iginstantfollowers.com/Log_in_Instagram.php', [[None, None]]]
['https://unfollowplus.com/oauth.php', [[None, None]]]
['http://www.verify.chase.billing.info.igualdad.cl/', [[None, None]]]
['http://wenaxe.com/aa/login.php?l=_JeHFUq_VJOXK0QWHtoGYDw1774256418&amp;fid.13InboxLight.aspxn.1774256418&amp;fid.125289964252813InboxLight99642_Product-email&amp;email=', [[None, None]]]
['http://wenaxe.com/aa/surf2.php', [[None, None]]]
['http://wenaxe.com/aa', [[None, None]]]
['http://wenaxe.com/aa/login.php?l=_JeHFUq_VJOXK0QWHtoGYDw1774256418&fid.13InboxLight.aspxn.1774256418&fid.125289964252813InboxLight99642_Product-email&email=', [[None, None]]]
['http://aruba-iv.blogspot.com/', [[None, None]]]
['https://aruba-iv.blogspot.com/', [[None, None]]]
['https://aruba-iv.blogspot.com/?m=1', [[None, None]]]
['http://itukaskstore.com/', [[None, None]]]
['http://ocidadaobrasileiro.com/', [[None, None]]]
['http://prepperslistreview.com/wp-admin

['https://realchef.com.au/wp-content/plugins/envato-market/iam/postale.activation%203/', [[None, None]]]
['https://realchef.com.au/wp-content/plugins/envato-market/iam/postale.activation%203', [[None, None]]]
['http://huntingtononline.ddns.info/suntrust/', [[None, None]]]
['http://ladycavendishdiary.com/office/doc/doc/main.html?accesstofile=true&amp;fileaccess=69803&amp;encryptedcookie=f56d71db3a63dc6eb4b8c77247eb2987&amp;u=1f48ce5792be955ef999e1c98354ec2e&amp;connecting=b6b59f3b13262a2a9c9dab1ed11515a3&amp;phaseaccess=de76026e2e2f4bfcf6222a0af1854203&amp;p=da467ece3cd1d85545b21940cec85864', [[None, None]]]
['http://ladycavendishdiary.com/office/doc/doc/main.html?accesstofile=valid&amp;fileaccess=14268&amp;encryptedcookie=00aae848e3754f2625a1408ff899e671&amp;u=922fe913e190ef3c493b1927dca45399&amp;connecting=358d7f6e07038383211c7171ead99f9c&amp;phaseaccess=f7931c614fc74a0dce31f22a7b70673e&amp;p=c20241929baaaf9e7e0a45cace346404', [[None, None]]]
['http://ladycavendishdiary.com/office/doc

['http://www.ff-oberoetzdorf.de/aktuell/FCKeditor/editor/filemanager/browser/default/connectors/lasso/comfirme-demande-compte.php', [[None, None]]]
['https://howto.indecosoft.ro/wp-includes/fonts/upgrade.php', [[None, None]]]
['https://www.automationhub.pk/pay-untetct/cc/', [[None, None]]]
['http://faktor.rs/nasulaz/images/cp/index.php', [[None, None]]]
['http://www.atefnet.com/js/calendar/login/customer_center/customer-IDPP00C699/myaccount/signin', [[None, None]]]
['http://www.atefnet.com/js/calendar/login/customer_center/customer-IDPP00C699/myaccount/signin/', [[None, None]]]
['https://suspended-profiles.blogspot.com/', [[None, None]]]
['http://stolizaparketa.ru/wp-content/themes/twentyfifteen/css/read/chinavali/index.php?email=xxx@yyy.dabsol.co.uk/', [[None, None]]]
['http://stolizaparketa.ru/wp-content/themes/twentyfifteen/css/read/chinavali/index.php?email=xxx@yyy.dabsol.co.uk', [[None, None]]]
['http://stolizaparketa.ru/wp-content/themes/twentyfifteen/css/read/chinavali/index.php

['sputnik-georgia.ru', [[None, None]]]
['ptcl.com.pk', [[None, None]]]
['homecredit.co.in', [[None, None]]]
['websitetooltester.com', [[None, None]]]
['dutaindoxxi.net', [[None, None]]]
['echodnia.eu', [[None, None]]]
['metabattle.com', [[None, None]]]
['raz-plus.com', [[None, None]]]
['f2pool.com', [[None, None]]]
['cashrewards.com.au', [[None, None]]]
['locopelis.com', [[None, None]]]
['ddl.to', [[None, None]]]
['codedereduc.fr', [[None, None]]]
['imbc.com', [[None, None]]]
['dogeclick.com', [[None, None]]]
['mann-ivanov-ferber.ru', [[None, None]]]
['kinepolis.be', [[None, None]]]
['bluecorona.com', [[None, None]]]
['tvfplay.com', [[None, None]]]
['gamato-movies.gr', [[None, None]]]
['photopin.com', [[None, None]]]
['anagnostis.org', [[None, None]]]
['91wii.com', [[None, None]]]
['filmfare.com', [[None, None]]]
['manybooks.net', [[None, None]]]
['lawdepot.com', [[None, None]]]
['glovoapp.com', [[None, None]]]
['mangahato.com', [[None, None]]]
['myaccountaccess.com', [[None, None]]]
[

['capital.fr', [[None, None]]]
['globfone.com', [[None, None]]]
['searchfort.online', [[None, None]]]
['popmatters.com', [[None, None]]]
['bcv.org.ve', [[None, None]]]
['pichak.net', [[None, None]]]
['fayerwayer.com', [[None, None]]]
['dollarshaveclub.com', [[None, None]]]
['ichacha.net', [[None, None]]]
['malaymail.com', [[None, None]]]
['porsline.ir', [[None, None]]]
['daemon-tools.cc', [[None, None]]]
['tunebat.com', [[None, None]]]
['graspgold.com', [[None, None]]]
['educationcity.com', [[None, None]]]
['armeniasputnik.am', [[None, None]]]
['rapha.cc', [[None, None]]]
['cissamagazine.com.br', [[None, None]]]
['natelinha.uol.com.br', [[None, None]]]
['natureetdecouvertes.com', [[None, None]]]
['hexus.net', [[None, None]]]
['dongyoungsang.club', [[None, None]]]
['cheapoair.ca', [[None, None]]]
['zakupka.com', [[None, None]]]
['text-pesni.com', [[None, None]]]
['osde.com.ar', [[None, None]]]
['hrmshry.nic.in', [[None, None]]]
['androidforums.com', [[None, None]]]
['unmc.edu', [[None, 

['erobi.net', [[None, None]]]
['huaxunchina.cn', [[None, None]]]
['jhnews.com.cn', [[None, None]]]
['ktpis.com', [[None, None]]]
['poropro.top', [[None, None]]]
['qqct58.com', [[None, None]]]
['rrcg.cn', [[None, None]]]
['ty360.com', [[None, None]]]
['wellwig.com', [[None, None]]]
['yynys.com', [[None, None]]]
['optuszoo.com.au', [[None, None]]]
['architrade.com', [[None, None]]]
['quickanddirtytips.com', [[None, None]]]
['vodafone.com', [[None, None]]]
['enabbaladi.net', [[None, None]]]
['codota.com', [[None, None]]]
['littlestyleshop.com', [[None, None]]]
['sevenstore.com', [[None, None]]]
['myt.mu', [[None, None]]]
['seleck.cc', [[None, None]]]
['voirfilms.al', [[None, None]]]
['expireddomains.net', [[None, None]]]
['likejapan.com', [[None, None]]]
['tudu.com.br', [[None, None]]]
['underarmour.com.br', [[None, None]]]
['truecorp.co.th', [[None, None]]]
['pornolulz-e.com', [[None, None]]]
['coveralia.com', [[None, None]]]
['karvy.com', [[None, None]]]
['a10.com', [[None, None]]]
['me

['okexcoin.vip', [[None, None]]]
['proporta.co.uk', [[None, None]]]
['qureo.jp', [[None, None]]]
['sergiorossi.com', [[None, None]]]
['shouraikai.jp', [[None, None]]]
['speedcoaching.co.jp', [[None, None]]]
['syosetu.org', [[None, None]]]
['toshinjyuken.co.jp', [[None, None]]]
['truffleshuffle.co.uk', [[None, None]]]
['urbanshooz.fr', [[None, None]]]
['y-classy.com', [[None, None]]]
['youravhost.net', [[None, None]]]
['yutaka-kotsu.co.jp', [[None, None]]]
['dailyqudrat.pk', [[None, None]]]
['gepuwang.net', [[None, None]]]
['pfind.com', [[None, None]]]
['nepallive.com', [[None, None]]]
['7y7.com', [[None, None]]]
['dayoo.com', [[None, None]]]
['flightclub.cn', [[None, None]]]
['nyasama.com', [[None, None]]]
['sbar.com.cn', [[None, None]]]
['tom.com', [[None, None]]]
['bath.ac.uk', [[None, None]]]
['projectfreetv.ag', [[None, None]]]
['tune.com', [[None, None]]]
['peugeot.es', [[None, None]]]
['mytracking.net', [[None, None]]]
['emby.media', [[None, None]]]
['toeslagen.nl', [[None, None]

In [12]:
dfPred = df
dfPred = dfPred.drop("url",1)
print(df)
con.close()

                                                     url  classification  \
0      https://cpakaren.com/bin-cgi/second-wave/GetMy...               1   
1      https://cpakaren.com/bin-cgi/second-wave/GetMy...               1   
2                             http://www.s5s.uno/?iR1087               1   
3                            https://ww2.yuriybobak.com/               1   
4      http://rakuten.co.jp.gfrhgfdxvfdegfcasdfesdght...               1   
...                                                  ...             ...   
23443                                            kef.com               0   
23444                                      wisebread.com               0   
23445                                           lsac.org               0   
23446                                      admagazine.ru               0   
23447                                    adventistas.org               0   

       dash  domainLength  underscore  at  slash  equals   http    www  ...  \
0       

In [13]:
x1 = dfPred.iloc[:,1:]
dfPred = dfPred.sample(frac=1).reset_index(drop=True)
x = dfPred.iloc[:,1:]
y = dfPred.iloc[:,0]
print(x)
print(y)
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                    test_size=0.2, random_state=12421)

#CELLS UP TO HERE ARE NEEDED TO RUN, THE REST ARE FOR SPECIFIC AI ALGORITHMS/OPTIMISATION ALGORITHMS

       dash  domainLength  underscore  at  slash  equals   http    www  dot  \
0         0             7           0   0      0       0  False  False    1   
1         0             8           0   0      0       0  False  False    1   
2         1            10           0   0      2       0  False  False    1   
3         3             8           0   0      1       0  False  False    1   
4         0            14           0   0      0       0  False  False    1   
...     ...           ...         ...  ..    ...     ...    ...    ...  ...   
23443     0             4           0   0      0       0  False  False    1   
23444     0             4           0   0      3       0  False  False    2   
23445     0            11           0   0      0       0  False  False    1   
23446     0            14           0   0      0       0  False  False    1   
23447     1            12           0   0      1       0  False  False    1   

       digit  ...  comTrTld  ipInUrl  linksToNowher

In [10]:
"""
def psoObjectiveFuncGeneric(mask, alpha=0.5):
    colLen = len(x.columns)
    if np.count_nonzero(mask) == 0:
        xTest = x
    else:
        xTest = x[:,mask==1]
#INSERT CLASSIFIER HERE
"""

'\ndef psoObjectiveFuncGeneric(mask, alpha=0.5):\n    colLen = len(x.columns)\n    if np.count_nonzero(mask) == 0:\n        xTest = x\n    else:\n        xTest = x[:,mask==1]\n#INSERT CLASSIFIER HERE\n'

In [18]:
#Required for all PSO and MFO optimisation
#ADAPTED FROM https://pyswarms.readthedocs.io/en/latest/examples/usecases/feature_subset_selection.html?highlight=feature%20subset
def psoObjectiveFuncLR(mask, alpha=0.5):
    colLen = len(x.columns)
    xConv = x.to_numpy()
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
    md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
    md.fit(x_test,y)
    performance = md.predict(x_test)
    performance = accuracy_score(y, performance)
    result = (alpha*(1.0-performance)+(1.0-alpha)*(1-(x_test.shape[1]/colLen)))
    return result

def psoObjectiveFuncSVM(mask, alpha=0.5):
    colLen = len(x.columns)
    xConv = x.to_numpy()
#     print(mask)
#     print(xConv)
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
    md = svm.LinearSVC(C=1.1,penalty='l1',max_iter=10000, dual=False)
    md.fit(x_test,y)
    performance = md.predict(x_test)
    performance = accuracy_score(y, performance)
    result = (alpha*(1.0-performance)+(1.0-alpha)*(1-(x_test.shape[1]/colLen)))
    return result

def MFOObjectiveFuncLR(xPassed, alpha=0.5):
    colLen = len(x.columns)
    mask = np.reshape(xPassed,(1,-1))
    mask = np.round(mask)
    mask = mask[0]
    xConv = x.to_numpy()
#     print(xConv)
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
#     print(x_test)
    md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
    md.fit(x_test,y)
    performance = md.predict(x_test)
    performance = accuracy_score(y, performance)
    result = (alpha*(1.0-performance)+(1.0-alpha)*(1-(x_test.shape[1]/colLen)))
    return result

def MFOObjectiveFuncSVM(xPassed, alpha=0.5):
    colLen = len(x.columns)
    mask = np.reshape(xPassed,(1,-1))
    mask = np.round(mask)
    mask = mask[0]
    xConv = x.to_numpy()
#     print(xConv)
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
#     print(x_test)
    md = svm.LinearSVC(C=1.1,penalty='l1',max_iter=10000, dual=False)
    md.fit(x_test,y)
    performance = md.predict(x_test)
    performance = accuracy_score(y, performance)
    result = (alpha*(1.0-performance)+(1.0-alpha)*(1-(x_test.shape[1]/colLen)))
    return result

def MFOObjectiveFuncRF(xPassed, alpha=1):
    colLen = len(x.columns)
    mask = np.reshape(xPassed,(1,-1))
    mask = np.round(mask)
    mask = mask[0]
    xConv = x.to_numpy()
#     print(xConv)
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
#     print(x_test)
    md = RandomForestClassifier(n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1)
#     md.fit(x_test,y)
    performance = mean(cross_val_score(md,x,y,cv=5,scoring="f1"))
    print(performance)
    result = -performance
    return result


def psoObjectiveFuncRF(mask, alpha=1):
#     print(mask)
    colLen = len(x.columns)
    xConv = x.to_numpy()
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
    md = RandomForestClassifier(bootstrap = True, criterion='entropy', n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1)
    print(x_test)
    performance = mean(cross_val_score(md,x,y,cv=5,scoring="f1"))
    print(performance)
    result = -performance
    return result


def psoObjectiveFuncNN(mask, alpha=0.5):
    colLen = x_test.shape[1]
    md = Sequential()
    md.add(Dense(colLen,input_shape=(colLen,)))
    #needs to be low enough neurons to limit overfitting, but high enough to be useful
    md.add(BatchNormalization())
    md.add(Activation('relu'))

    md.add(Dense(371))
    md.add(BatchNormalization())
    md.add(Activation('relu'))
    md.add(Dropout(0.25))

    md.add(Dense(395))
    md.add(BatchNormalization())
    md.add(Activation('relu'))
    md.add(Dropout(0.41))

    md.add(Dense(52))
    md.add(BatchNormalization())
    md.add(Activation('relu'))
    md.add(Dropout(0.31))
    md.add(Dense(1, activation='sigmoid'))
    #normalises data into a probability distribution
    md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
    md.fit(x_test,y,68,10)
    performance = md.predict(x_test)
    performance = np.round(performance)
    performance = accuracy_score(y, performance)
    result = (alpha*(1.0-performance)+(1.0-alpha)*(1-(x_test.shape[1]/colLen)))
    return result

def MFOObjectiveFuncNN(xPassed, alpha=0.5):
    mask = np.reshape(xPassed,(1,-1))
    mask = np.round(mask)
    mask = mask[0]
    xConv = x.to_numpy()
    if np.count_nonzero(mask) == 0:
        x_test = xConv
    else:
        x_test = xConv[:,mask==1]
    colLen = x_test.shape[1]
    md = Sequential()
    md.add(Dense(colLen,input_shape=(colLen,)))
    #needs to be low enough neurons to limit overfitting, but high enough to be useful
    md.add(BatchNormalization())
    md.add(Activation('relu'))

    md.add(Dense(371))
    md.add(BatchNormalization())
    md.add(Activation('relu'))
    md.add(Dropout(0.25))

    md.add(Dense(395))
    md.add(BatchNormalization())
    md.add(Activation('relu'))
    md.add(Dropout(0.41))

    md.add(Dense(52))
    md.add(BatchNormalization())
    md.add(Activation('relu'))
    md.add(Dropout(0.31))
    md.add(Dense(1, activation='sigmoid'))
    #normalises data into a probability distribution
    md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
    md.fit(x_test,y,68,10)
    performance = md.predict(x_test)
    performance = np.round(performance)
    performance = accuracy_score(y, performance)
    result = (alpha*(1.0-performance)+(1.0-alpha)*(1-(x_test.shape[1]/colLen)))
    return result


def loopPso(x,alpha=1):
    particleCount = x.shape[0]
# The below need to be changed to fit the model being tested, really this should
# be altered to be more robust.
    loss = [psoObjectiveFuncRF(x[i],alpha) for i in range(particleCount)]
    return np.array(loss)



In [67]:
#Adapted from https://github.com/thieunguyen5991/mealpy/blob/master/mealpy/swarm_based/MFO.py

md =BaseMFO(MFOObjectiveFuncSVM,47,(0,1),100, 100)
pos, fit, loss = md._train__()
print(fit)          

KeyError: 0

In [ ]:
print(pos)    
mask = np.reshape(pos/2,(1,-1))

mask = np.round(mask)
mask = mask[0]
print(mask)

In [ ]:
mask = np.reshape(pos/2,(1,-1))
mask = np.round(mask)
mask = mask[0]
xConv = x.to_numpy()
xFeatures = xConv[:,mask==1]
md = RandomForestClassifier(bootstrap = True, criterion='entropy', n_estimators = 10000,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))
# [0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
#  1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
# 0.9360713067212555

In [ ]:
#BASE MODEL LINEAR REGRESSION accuracy:0.6789
md = LogisticRegression(random_state=4324, solver='lbfgs', multi_class='ovr', max_iter=100000)
md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#TPE optimisation LINEAR REGRESSION accuracy:
#0.05% less accurate than base model
#TODO SET UP PARAMS
space = {'solver' : hp.choice("solver",['newton-cg','lbfgs','sag','saga']),
         'class_weight' : hp.choice("class_weight",[None,'balanced']),
         'C' : hp.choice("C",[0.8,0.9,1.0,1.1,1.2])
         }

def objective(space):
    md = LogisticRegression(solver=space['solver'],
                                class_weight=space['class_weight'],
                                C=space["C"],
                                n_jobs=-1)
    accuracy = cross_val_score(md,X_train,y_train, cv=10).mean()
    return{'loss': -accuracy,'status':STATUS_OK}

best = fmin(fn=objective,
            space=space,
            algo= tpe.suggest,
            max_evals = 40,
            trials = Trials())
#there's only 40 combinations of parameters so no reason to do more.

best

In [ ]:
md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#Genetic optimisation LINEAR REGRESSION accuracy:

space = {'solver' : ['newton-cg','lbfgs','sag','saga'],
         'class_weight' : [None,'balanced'],
         'C' :[0.8,0.9,1.0,1.1,1.2]
         }

md = TPOTClassifier(generations = 15, population_size=40, offspring_size=40, verbosity = 2,
                   config_dict = {'sklearn.linear_model.LogisticRegression':space},
                   cv = 10, scoring = 'accuracy', n_jobs=-1)

md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
options = {'c1':0.5,'c2':0.5,'w':0.5,'k':33,'p':2}
dimensions = len(x.columns)
optimiser = pso.discrete.BinaryPSO(n_particles=33, dimensions=dimensions, options=options)
cost, pos = optimiser.optimize(loopPso,iters = 30)

In [ ]:
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))


In [ ]:
md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
import GAFO
test = GAFO.makeGene(0.01,100,100)
test.fit(md,"classification",x,y)
print(test.results())

In [ ]:
pos = test.results()[0]
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [ ]:
md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
pos, fit, loss = md._train__()
print(fit)          
performance = md.predict(x_test)
performance = accuracy_score(y, performance)

In [ ]:
md =BaseMFO(MFOObjectiveFuncLR,47,(0,1),25, 25)
pos, fit, loss = md._train__()
print(fit)       

In [ ]:
print(pos)    
mask = np.reshape(pos/2,(1,-1))

mask = np.round(mask)
mask = mask[0]
print(mask)

In [ ]:
mask = np.reshape(pos,(1,-1))
mask = np.round(mask)
mask = mask[0]
xConv = x.to_numpy()
xFeatures = xConv[:,mask==1]
md = LogisticRegression(random_state=4324, solver='newton-cg', multi_class='ovr', max_iter=100000)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [ ]:
#BASE MODEL SVM accuracy:

X_train = normalize(X_train)
X_test = normalize(X_test)
md = svm.LinearSVC()
md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))
prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#TPE optimisation SVM accuracy:
#code adpated from https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d
X_train = normalize(X_train)
X_test = normalize(X_test)
space = {'penalty' : hp.choice("penalty",['l1','l2']),
         'class_weight' : hp.choice("class_weight",[None,'balanced']),
         'C' : hp.choice("C",[0.8,0.9,1.0,1.1,1.2])
         }

def objective(space):
    md = svm.LinearSVC(penalty=space['penalty'],
                                class_weight=space['class_weight'],
                                C=space["C"], dual=False, max_iter=10000)
    accuracy = cross_val_score(md,X_train,y_train, cv=10).mean()
    return{'loss': -accuracy,'status':STATUS_OK}

best = fmin(fn=objective,
            space=space,
            algo= tpe.suggest,
            max_evals = 20,
            trials = Trials())

best

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)
md = svm.LinearSVC(C=1.1,penalty='l1',max_iter=10000, dual=False)
md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))
prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#Genetic optimisation SVM accuracy:
X_train = normalize(X_train)
X_test = normalize(X_test)
space = {'penalty' : ['l1','l2'],
         'class_weight' :[None,'balanced'],
         'C' : [0.8,0.9,1.0,1.1,1.2]
         }

md = TPOTClassifier(generations = 15, population_size=100, offspring_size=100, verbosity = 2,
                   config_dict = {'sklearn.svm.LinearSVC':space},
                   cv = 10, scoring = 'accuracy', n_jobs=-1)

md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
options = {'c1':0.5,'c2':0.5,'w':0.9,'k':20,'p':2}
dimensions = len(x.columns)
optimiser = pso.discrete.BinaryPSO(n_particles=100, dimensions=dimensions, options=options)


cost, pos = optimiser.optimize(loopPso,iters = 100)


In [ ]:
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
md = svm.LinearSVC(C=1.1,penalty='l1',max_iter=10000, dual=False)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))


In [ ]:
md = svm.LinearSVC(C=1.1,penalty='l1',max_iter=10000, dual=False)
import GAFO
test = GAFO.makeGene(0.001,100,100)
test.fit(md,"classification",x,y)
print(test.results())

In [ ]:
pos = test.results()[0]

In [ ]:
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
md = svm.LinearSVC(C=1.1,penalty='l1',max_iter=10000, dual=False)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [ ]:
#BASE MODEL RANDOM FOREST accuracy:96.02%

md = RandomForestClassifier(n_estimators = 10000, verbose=1, n_jobs = -1)
md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")




In [ ]:
#TPE optimisation RANDOM FOREST accuracy:
#code adpated from https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d
space = {'criterion' : hp.choice("criterion",['entropy','gini']),
         'max_depth' : hp.choice("depth",[10,25,50,100,250,500,1000]),
         'min_samples_split' : hp.choice("minSamplesSplit",[2,3,4,5,6,7,8,9,10,11,12]),
         'min_samples_leaf' : hp.choice("minSamplesLeaf",[1,2,3,4,5,6,7,8,9,10]),
         'max_features' : hp.choice("maxFeat",["auto","log2",None]),
         'bootstrap':hp.choice("bootstrap",[True,False]),
         'n_estimators':hp.choice("estimators",[100,250,500])}

def objective(space):
    md = RandomForestClassifier(criterion=space['criterion'],
                                max_depth=space['max_depth'],
                                min_samples_split=space["min_samples_split"],
                                min_samples_leaf=space["min_samples_leaf"],
                                max_features=space["max_features"],
                                bootstrap=space["bootstrap"],
                                n_estimators=space["n_estimators"],
                                n_jobs=-1)
    accuracy = cross_val_score(md,X_train,y_train, cv=10).mean()
    return{'loss': -accuracy,'status':STATUS_OK}

best = fmin(fn=objective,
            space=space,
            algo= tpe.suggest,
            max_evals = 100,
            trials = Trials())

best

In [ ]:
#Testing the optimised parameters with a larger number of estimators
md = RandomForestClassifier(bootstrap = True, criterion='entropy', n_estimators = 10000,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, verbose=1, n_jobs = -1)
md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")


In [ ]:
#Genetic algorithm optimisation RANDOM FOREST accuracy:
space = {'criterion' : ['entropy','gini'],
         'max_depth' :[10,25,50,100,250,500,1000],
         'min_samples_split' : [2,3,4,5,6,7,8,9,10,11,12],
         'min_samples_leaf' : [1,2,3,4,5,6,7,8,9,10],
         'max_features' : ["auto","log2",None],
         'bootstrap':[True,False],
         'n_estimators':[100]}

md = TPOTClassifier(generations = 15, population_size=15, offspring_size=15, verbosity = 2,
                   config_dict = {'sklearn.ensemble.RandomForestClassifier':space},
                   cv = 10, scoring = 'accuracy', n_jobs=-1)
# large number of changes means large number of comparisons to do

md.fit(X_train,y_train)
prediction = md.predict(X_test)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [68]:
options = {'c1':0.5,'c2':0.5,'w':0.9,'k':20,'p':2}
dimensions = len(x.columns)
optimiser = pso.discrete.BinaryPSO(n_particles=40, dimensions=dimensions, options=options)

cost, pos = optimiser.optimize(loopPso,iters = 40)


2020-10-15 06:07:13,763 - pyswarms.discrete.binary - INFO - Optimize for 40 iters with {'c1': 0.5, 'c2': 0.5, 'w': 0.9, 'k': 20, 'p': 2}
pyswarms.discrete.binary:   0%|          |0/40

[[0 0 0 ... 1259 1 0]
 [0 0 0 ... 13 1 0]
 [0 0 0 ... -1 2 0]
 ...
 [0 0 0 ... 1068 2 0]
 [0 0 0 ... 24 2 0]
 [0 0 9 ... 24 8 0]]
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 2 0 0]
 ...
 [0 0 0 ... 2 1 0]
 [0 0 0 ... 2 0 0]
 [1 9 0 ... 8 0 0]]
[[0 0 0 ... 18 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... -1 2 0]
 ...
 [0 0 0 ... 12 2 0]
 [0 0 0 ... 0 2 0]
 [1 0 0 ... 0 8 0]]
[[0 10 0 ... 1259 1 0]
 [0 8 0 ... 13 1 0]
 [0 7 0 ... -1 2 0]
 ...
 [0 15 0 ... 1068 2 0]
 [0 10 0 ... 24 2 0]
 [1 15 9 ... 24 8 0]]
[[0 0 0 ... 0.0 0 0]
 [0 0 0 ... 0.0 0 0]
 [0 0 0 ... 0.0 0 0]
 ...
 [0 0 0 ... 0.959349593495935 1 0]
 [0 0 0 ... 0.0 0 0]
 [1 0 0 ... 0.0 0 0]]
[[0 0 False ... 18 7776000 0]
 [0 0 False ... 0 7776000 0]
 [0 0 False ... -1 -1 0]
 ...
 [0 0 False ... 12 -1 1]
 [0 0 False ... 0 -1 0]
 [0 9 False ... 0 31579200 0]]
[[0 1 0 ... 18 1259 0]
 [0 1 0 ... 0 13 0]
 [0 1 0 ... -1 -1 0]
 ...
 [0 1 0 ... 12 1068 1]
 [0 1 0 ... 0 24 0]
 [9 1 0 ... 0 24 0]]
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0

pyswarms.discrete.binary:   2%|▎         |1/40, best_cost=0.00281

[[0 0 0 ... 1259 0 0]
 [0 0 0 ... 13 0 0]
 [0 0 0 ... -1 0 0]
 ...
 [0 0 0 ... 1068 1 0]
 [0 0 0 ... 24 0 0]
 [1 0 9 ... 24 0 0]]
[[0 0 0 ... 1259 0 0]
 [0 0 0 ... 13 0 0]
 [0 0 0 ... -1 0 0]
 ...
 [0 0 0 ... 1068 1 0]
 [0 0 0 ... 24 0 0]
 [1 0 9 ... 24 0 0]]
[[0 10 0 ... 1259 0 0]
 [0 8 0 ... 13 0 0]
 [0 7 0 ... -1 0 0]
 ...
 [0 15 0 ... 1068 1 0]
 [0 10 0 ... 24 0 0]
 [1 15 0 ... 24 0 0]]
[[0 0 0 ... 7776000 0 0]
 [0 0 0 ... 7776000 0 0]
 [0 0 0 ... -1 0 0]
 ...
 [0 0 0 ... -1 1 0]
 [0 0 0 ... -1 0 0]
 [1 0 9 ... 31579200 0 0]]
[[0 0 0 ... 1259 0 0]
 [0 0 0 ... 13 0 0]
 [0 0 0 ... -1 0 0]
 ...
 [0 0 0 ... 1068 1 0]
 [0 0 0 ... 24 0 0]
 [1 0 9 ... 24 0 0]]
[[0 0 0 ... 1259 0 0]
 [0 0 0 ... 13 0 0]
 [0 0 0 ... -1 0 0]
 ...
 [0 0 0 ... 1068 1 0]
 [0 0 0 ... 24 0 0]
 [1 0 9 ... 24 0 0]]
[[0 10 0 ... 7776000 0 0]
 [0 8 0 ... 7776000 0 0]
 [0 7 0 ... -1 0 0]
 ...
 [0 15 0 ... -1 1 0]
 [0 10 0 ... -1 0 0]
 [1 15 0 ... 31579200 0 0]]
[[0 10 0 ... 1259 0 0]
 [0 8 0 ... 13 0 0]
 [0 7 0 ... -1 

pyswarms.discrete.binary:   5%|▌         |2/40, best_cost=0.00256

...

pyswarms.discrete.binary:  95%|█████████▌|38/40, best_cost=0.00222

[[0 0 0 ... 0.0 7776000 0]
 [0 0 0 ... 0.0 7776000 0]
 [0 0 0 ... 0.0 -1 0]
 ...
 [0 0 0 ... 0.024390243902439025 -1 0]
 [0 0 0 ... 0.0 -1 0]
 [0 9 0 ... 0.0 31579200 0]]
[[0 0 0 ... 0.0 7776000 0]
 [0 0 0 ... 0.0 7776000 0]
 [0 0 0 ... 0.0 -1 0]
 ...
 [0 0 0 ... 0.024390243902439025 -1 0]
 [0 0 0 ... 0.0 -1 0]
 [1 0 9 ... 0.0 31579200 0]]
[[0 0 0 ... 18 7776000 0]
 [0 0 0 ... 0 7776000 0]
 [0 0 0 ... -1 -1 0]
 ...
 [0 0 0 ... 12 -1 0]
 [0 0 0 ... 0 -1 0]
 [1 0 9 ... 0 31579200 0]]
[[0 0 0 ... 0.0 7776000 0]
 [0 0 0 ... 0.0 7776000 0]
 [0 0 0 ... 0.0 -1 0]
 ...
 [0 0 0 ... 0.024390243902439025 -1 0]
 [0 0 0 ... 0.0 -1 0]
 [1 0 9 ... 0.0 31579200 0]]
[[0 0 0 ... 7776000 1 0]
 [0 0 0 ... 7776000 1 0]
 [0 0 0 ... -1 2 0]
 ...
 [0 0 0 ... -1 2 0]
 [0 0 0 ... -1 2 0]
 [1 0 9 ... 31579200 8 0]]
[[0 0 0 ... 18 7776000 0]
 [0 0 0 ... 0 7776000 0]
 [0 0 0 ... -1 -1 0]
 ...
 [0 0 0 ... 12 -1 0]
 [0 0 0 ... 0 -1 0]
 [1 0 9 ... 0 31579200 0]]
[[0 0 0 ... 7776000 1 0]
 [0 0 0 ... 7776000 1 0]
 [0 0

pyswarms.discrete.binary:  98%|█████████▊|39/40, best_cost=0.00222

[[0 10 0 ... 1259 7776000 0]
 [0 8 0 ... 13 7776000 0]
 [0 7 0 ... -1 -1 0]
 ...
 [0 15 0 ... 1068 -1 1]
 [0 10 0 ... 24 -1 0]
 [1 15 0 ... 24 31579200 0]]
[[0 10 0 ... 1259 7776000 0]
 [0 8 0 ... 13 7776000 0]
 [0 7 0 ... -1 -1 0]
 ...
 [0 15 0 ... 1068 -1 1]
 [0 10 0 ... 24 -1 0]
 [1 15 0 ... 24 31579200 0]]
[[0 10 0 ... 1259 7776000 0]
 [0 8 0 ... 13 7776000 0]
 [0 7 0 ... -1 -1 0]
 ...
 [0 15 0 ... 1068 -1 1]
 [0 10 0 ... 24 -1 0]
 [1 15 0 ... 24 31579200 0]]
[[0 10 0 ... 1259 7776000 0]
 [0 8 0 ... 13 7776000 0]
 [0 7 0 ... -1 -1 0]
 ...
 [0 15 0 ... 1068 -1 1]
 [0 10 0 ... 24 -1 0]
 [1 15 0 ... 24 31579200 0]]
[[0 10 0 ... 7776000 1 0]
 [0 8 0 ... 7776000 1 0]
 [0 7 0 ... -1 2 0]
 ...
 [0 15 0 ... -1 2 1]
 [0 10 0 ... -1 2 0]
 [1 15 0 ... 31579200 8 0]]
[[0 10 0 ... 1259 7776000 0]
 [0 8 0 ... 13 7776000 0]
 [0 7 0 ... -1 -1 0]
 ...
 [0 15 0 ... 1068 -1 1]
 [0 10 0 ... 24 -1 0]
 [1 15 0 ... 24 31579200 0]]
[[0 10 0 ... 1259 7776000 0]
 [0 8 0 ... 13 7776000 0]
 [0 7 0 ... -1 -1 0

pyswarms.discrete.binary: 100%|██████████|40/40, best_cost=0.00222
2020-10-15 06:24:42,816 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.0022176731490958224, best pos: [1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 0 0 1 1 1 1 0 1 1]


In [85]:
print(pos)

[1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 0 0 1 1 1 1 0 1 1]


In [15]:
mask =[]
pos = [1,1,1,0,1,0,0,0,0,1,0,0,1,1,1,1,1,0,1,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1,0,1,1]

for item in pos:
    if item>0:
        mask.append(1)
    else:
        mask.append(0)
print(mask)
xConv = x.to_numpy()
print(len(xConv[0]))
removals = []
for i in range(0,len(mask)-1):
    if mask[i] == 0:
        removals.append(i+1)

print(removals)
for removal in reversed(removals):
    xConv = np.delete(xConv,removal,1)
        
print(xConv)
# xFeatures = np.reshape(xConv,(-1,1))
xFeatures = pd.DataFrame(data=xConv)
print(xFeatures)
md = RandomForestClassifier(n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
accuracy = cross_val_score(md,x,y, cv=5).mean()
print(accuracy)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))
print(sklearn.metrics.recall_score(y,prediction))
print(sklearn.metrics.precision_score(y,prediction))

[1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1]
47
[4, 6, 7, 8, 9, 11, 12, 18, 20, 23, 24, 25, 26, 37, 39, 40, 45]
[[0 7 0 ... 63158399 5 0]
 [0 8 0 ... 7257600 6 0]
 [1 10 0 ... -1 2 0]
 ...
 [0 11 0 ... 31579200 8 0]
 [0 14 0 ... 31579200 8 0]
 [1 12 0 ... 31579200 9 0]]
      0   1  2  3  4  5  6  7  8  9   ... 20 21 22 23 24  25   26        27  \
0      0   7  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   0   41  63158399   
1      0   8  0  0  0  0  0  0  1  0  ...  0  0  0  0  0   0   12   7257600   
2      1  10  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  24   79        -1   
3      3   8  0  0  0  0  0  0  1  0  ...  0  0  0  0  0   0   12   7257600   
4      0  14  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  11   26  39096000   
...   ..  .. .. .. .. .. .. .. .. ..  ... .. .. .. .. ..  ..  ...       ...   
23443  0   4  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  35  292  31536000   


In [75]:
# df["prediction"] = prediction
# df.to_csv("results.csv")
with open("detectionModel2.pkl",'wb') as fileW:
    pickle.dump(md,fileW)

In [ ]:
md = RandomForestClassifier(n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1,verbose=2)
import GAFO
test = GAFO.makeGene(0.001,100,100)
test.fit(md,"classification",x,y)
print(test.results())

In [ ]:
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
md = RandomForestClassifier(n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1,verbose=2)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [21]:
md =BaseMFO(MFOObjectiveFuncRF,47,(0,1),40, 40)
pos, fit, loss = md._train__()
print(fit)          
predictions = md.predict(x_test)
performance = accuracy_score(y, predictions)

0.9945647739398543
0.9946772924098272
0.9948288926332315
0.9947144933848417
0.9948285362093654
0.9946782069814573
0.9947534193167032
0.994526888435025
0.9946400315244112
0.9946391370395951
0.9946417268504434
0.9945642917621024
0.9948286486568898
0.9946033599502059
0.994601835551504
0.9948288223740275
0.9947155174575695
0.9948290350230894
0.9947537896490041
0.9945657027829858
0.9947536161666064
0.9947907651935347
0.9946406887088919
0.9947523948426513
0.9946773209489148
0.9946778059917991
0.9946781035769636
0.9946391754399247
0.9946779170606336
0.9946398635732966
0.9948286232437559
0.9946778047161631
0.9946784064287817
0.9947163630306272
0.9946404604009368
0.994678175094398
0.9948281237222277
0.9947527925338998
0.9946406303425105
0.994827994821143
0.9946781485584261
0.9947160908073306
0.9945273600854678
0.994526450095043
0.9947537880478606
0.9945644770961519
0.9945634667014654
0.9945645197369408
0.9945649057834962
0.9946031027045962
0.9946781871800925
0.9946772943170226
0.994752407199607

0.9946404978263415
0.9946778450187524
0.9944517164405468
0.9948288188661044
0.9947908622563083
0.9947911506414273
0.9947150236802454
0.9946027483126182
0.9946027772715462
0.9945271764207745
0.994714933826545
0.9947529789747601
0.9945652709843159
0.9947529790475502
0.9946782056419016
0.9945277010318438
0.9947536933658431
0.994678830610742
0.9944514892867512
0.9947911203489
0.9945658726871051
0.9946784906617312
0.9947152799208927
0.994753234978819
0.994753550005538
0.9944887759335864
0.9947157074980592
0.994640275411635
0.9945643799529126
0.9946395919721566
0.9948290799333337
0.9946781487281259
0.9946412325257543
0.9946774228584663
0.9946400906196844
0.9947152934791269
0.9947909336596853
0.9947903947268489
0.9947906524611524
0.9946777940691357
0.9948285931867931
0.9946403908651641
0.994602604766426
0.9947530488212396
0.9947155055759701
0.9946771227552132
0.9947158617184048
0.9947151507573414
0.9947531502189437
0.9946785456862455
0.9947160618505815
0.9948670624953938
0.9946770652757964
0.

0.9946778065548755
0.9947151937814128
0.9947160183678477
0.9948283804571704
0.994828252323731
0.9947154075708363
0.9946768332993835
0.9947156935821988
0.9948292081834423
0.9946778620820981
0.994715278089226
0.9948675154098281
0.9947158650181969
0.9947154927914965
0.9948290786337398
0.9948296337175135
0.9947520245938226
0.994716588456135
0.9946027924661333
0.9947151216313473
0.994677778083189
0.9947907493408213
0.9947152759521004
0.9947904789430463
0.9946783451637953
0.9948296749498605
0.9948298896770327
0.9948282794609321
0.9946021754668338
0.9947915887282608
0.9946408016015104
0.9946408738622272
0.9947159620586199
0.9947154766750897
0.9948665927474825
0.9946788858196669
0.9947150091815478
0.9945650462411483
0.9947148517617876
0.9944894041237591
0.9947536875548896
0.9945653165430728
0.9948296179950061
0.9944517767104082
0.9947912472670777
0.9948293615278017
> Epoch: 8, Best fit: -0.9949793512677443
0.994753204190121
0.9947529787456063
0.9947920033890497
0.994602648530922
0.994865539495

0.9946779914009284
0.9947529804324774
0.9945639053378004
0.9948279359223585
0.9947536755581606
0.9945263644699677
0.9946779170046016
0.9946788019880257
0.9946023617665687
0.9947530226690514
0.9948660215676222
0.9946403632443527
0.994752580965908
0.9947531791266175
0.9947165316377519
0.9947538163874589
0.9947149076512959
0.9946786582179722
0.9946033445486818
0.9946025465861705
0.994639419582587
> Epoch: 12, Best fit: -0.9949794504759912
0.9947160346452317
0.994752951524624
0.9948657541066135
0.9947163586681876
0.9947911199857582
0.9946399624521841
0.9947536336187874
0.9946775344114415
0.994790251317728
0.9947165151588313
0.9945655551025303
0.9947909660480801
0.994677704836715
0.9947907796413297
0.9947165742222811
0.9946398604053674
0.9946030186753861
0.9947148806054287
0.9947544442565707
0.9949404555990402
0.9948658798540535
0.9946786148624307
0.9947156918979553
0.9947912060814268
0.9947912206229632
0.9947535908452858
0.9945271299825955
0.9946786028933466
0.9946031480914633
0.9947538615

0.9947533353986263
0.9947163303822363
0.9944889779951089
0.9947157183808378
0.9946026464496188
0.9946783595141712
0.9946389377128368
0.9947154336373044
0.9945274290214278
0.994754058255061
0.9947528214738753
0.9947156753297042
0.9947533495199369
0.9947542465354302
0.9944894181505349
0.9947913495896746
0.994715706248331
0.9947147510398561
0.9945260487345524
0.9946023620011114
0.9946771212438545
0.994715536272005
0.9947149234302357
0.9947907369054881
0.9946778081123528
0.994601779136635
0.9948287333296124
0.9948288635719301
0.994753207293893
0.9947905812054342
0.994790765142632
0.9947914320516761
0.9948286791543928
0.9946402915091137
0.9947157630107203
0.9947149677054532
0.9946778925542695
0.9946022060079156
0.9946020775726422
0.9947157328574862
0.9946406190010911
0.9947155199406642
0.9946410578891324
0.9947150778750685
0.9947541035690325
0.9947164862082183
0.99456330780772
0.9944887616532558
0.9947524233169939
0.9947155501268269
0.9947160462171402
0.9947907367572549
0.9947913617460002
0

...

0.9947529374950116
0.9946784906617312
0.9946409741282088
0.9947529356248982
0.9947154481360021
0.9947906058680694
0.994753519503385
0.9947161749200198
0.9946013085241835
0.994639618373128
0.9947158205280417
0.994790306712671
0.9945268761669257
0.9947519395349103
0.9945645197369408
0.9946388809335048
0.994677802647805
0.9946773651095749
0.9946402902088638
0.99464141396523
0.994678172958218
0.9947912363739535
0.994603100670734
0.9948285520901631
0.9946022938719318
0.9948289620973119
0.9947531508515841
0.9946401479521644
0.994715647022332
0.9946785732017228
0.9946028165491098
0.9945649864661927
0.9947530929006235
0.9947916916902433
0.9946398800846696
0.9947152080563486
0.9946782612607338
0.9947906227085704
0.9946777904412273
0.9947162321129012
0.9947896967215766
0.9947532064669644
0.9948669747893155
0.9947911216341876
0.9946785903300256
0.99464013466335
0.9946783138444191
> Epoch: 25, Best fit: -0.9950176080628279
0.9946778219777457
0.9947533204477692
0.9947153187607319
0.9947904222308916

0.9947155075886429
0.9947156217141914
0.9948289624299979
0.9946027773120156
0.9947529652219802
0.9948283194220918
0.9945638207781903
0.9946390795963974
0.9946402616589924
0.9945279163465426
0.9945637513922317
0.9946015365155337
0.9945270149398422
0.9947528500075742
0.9945646897914634
0.9945272424021052
0.9946772793253854
0.9946399353780534
0.9945286834570702
0.9947903247002903
0.9947519808721694
0.9947914766304692
> Epoch: 29, Best fit: -0.9950176080628279
0.9947902782947533
0.9946407134653423
0.9946398618852643
0.99467783310154
0.9947150369007031
0.9946403489855833
0.9947908353238312
0.9946780056427142
0.9946785456862455
0.9947166028845096
0.9947160338477239
0.9944897869912571
0.9948662080729332
0.9947156934124989
0.9946788427441088
0.9947152899788763
0.9946779052621451
0.9947920056722136
0.994602703225673
0.9946777074498304
0.9944892899541878
0.9946014937662554
0.9947912182548969
0.9946786315804906
0.9948288065045323
0.9946017343419539
0.994791263334703
0.9947533621655883
0.994677462

0.9947156918979553
0.9944518159286126
0.9946399457988377
0.9948287363935616
0.9946407590317979
0.9948287489527275
0.9948295890556155
0.9946395224909049
0.9946394217105523
0.9946406440102632
0.9946412322241523
0.9945266774873066
0.9947911056353823
0.9946403762884616
0.9946783305870042
0.9948296728893322
0.9947536637230241
0.9946777924335649
0.9946406335829048
0.9946027322418056
0.9946024026184965
0.9946398023259232
0.9948286520974591
0.9947145381976356
0.9947534762992173
0.9946024639919362
0.9948279252523092
0.9946772346369162
0.9945643742443082
0.994488816766935
0.99475363397716
0.994677761891356
0.994677747468187
0.9947152653441902
0.9946398204907185
0.9946393085406161
0.9947157344640968
0.994790976889566
0.994564507275408
0.9945647900416625
0.9947904953783587
0.9948289912530772
0.994828763375982
0.9946403472762677
0.9946776221506778
0.9947534644998003
0.994715532415868
0.9947527949284387
0.9946407038075152
0.9946779070980873
0.9944877797742977
0.994791136059442
0.9947156752898355
0.9

0.9946395754803248
0.9945653423682981
0.9945280698454754
0.9948670208285828
0.9947527495401974
0.994452838675157
0.9947152207875655
0.9947532617042893
0.994715308125475
0.994754088734786
0.994791378599615
0.9946777504898302
0.9947529070103673
0.9947156753297042
0.9945645197564646
0.9947158209210358
0.9944899970674695
0.9945652751145433
0.99464053358432
0.9947533942276355
0.9947536611760734
0.9947528500075742
0.9947538329907124
0.9946024026184965
0.9946780767883333
0.994715135010008
0.9945262916113071
0.9947146102740879
0.9946780751488626
0.9947162016257247
0.9948275509886257
0.9946021754668338
0.9947155184325196
0.9946774790845255
0.9947158362902239
0.9946024744952112
0.9947531940988007
0.9947904629660363
0.9945271473809488
0.9946791029753036
0.9946391222512266
0.9947908338859365
0.9947528099159265
0.9947915333390039
0.9946769909789205
0.9945649172944709
0.9948671728578211
0.994602250120547
0.9946774886755185
0.9947911627027801
0.9946778187511642
0.994639877689038
0.9946772205527028
0.

AttributeError: 'BaseMFO' object has no attribute 'predict'

In [ ]:
print(performance)

In [23]:
print(pos)
print(fit)

[ 0.58939667 -0.05556761  0.59611607 -0.32778644  1.14564315 -0.12002549
  0.88324228  0.78178982 -1.13229854 -0.82444223  0.55139348 -0.11687051
 -0.3269125  -0.30062043 -1.26049449  0.96801271  1.38090503  0.55303204
  1.08812877  0.13502671  0.16873672  0.37954518  1.05801082  0.25264412
  0.08914806  1.22926155  0.58837553 -0.23241274 -0.11208043 -0.47410068
  0.1708531   0.37782585  0.21683915  0.02793357  2.6285607   0.88284562
  0.18455946  0.33371864  0.93014064 -0.04410969  1.48401678  0.62459714
  1.49915738  0.9862859   0.97949014 -1.28368376  0.57224673]
-0.9950176080628279


In [32]:
print(pos)
mask = []
mask = np.reshape(pos/2,(1,-1))
mask = np.round(mask)
mask = mask[0]
print(mask)

[ 0.58939667 -0.05556761  0.59611607 -0.32778644  1.14564315 -0.12002549
  0.88324228  0.78178982 -1.13229854 -0.82444223  0.55139348 -0.11687051
 -0.3269125  -0.30062043 -1.26049449  0.96801271  1.38090503  0.55303204
  1.08812877  0.13502671  0.16873672  0.37954518  1.05801082  0.25264412
  0.08914806  1.22926155  0.58837553 -0.23241274 -0.11208043 -0.47410068
  0.1708531   0.37782585  0.21683915  0.02793357  2.6285607   0.88284562
  0.18455946  0.33371864  0.93014064 -0.04410969  1.48401678  0.62459714
  1.49915738  0.9862859   0.97949014 -1.28368376  0.57224673]
[ 0. -0.  0. -0.  1. -0.  0.  0. -1. -0.  0. -0. -0. -0. -1.  0.  1.  0.
  1.  0.  0.  0.  1.  0.  0.  1.  0. -0. -0. -0.  0.  0.  0.  0.  1.  0.
  0.  0.  0. -0.  1.  0.  1.  0.  0. -1.  0.]


In [105]:
pos=[ 0.58939667, -0.05556761,  0.59611607, -0.32778644,  1.14564315, -0.12002549,
  0.88324228,  0.78178982, -1.13229854, -0.82444223,  0.55139348, -0.11687051,
 -0.3269125,  -0.30062043, -1.26049449,  0.96801271,  1.38090503,  0.55303204,
  1.08812877,  0.13502671,  0.16873672,  0.37954518,  1.05801082,  0.25264412,
  0.08914806,  1.22926155,  0.58837553, -0.23241274, -0.11208043, -0.47410068,
  0.1708531,   0.37782585,  0.21683915,  0.02793357,  2.6285607,   0.88284562,
  0.18455946,  0.33371864,  0.93014064, -0.04410969,  1.48401678,  0.62459714,
  1.49915738,  0.9862859,   0.97949014, -1.28368376,  0.57224673]
mask =[]
for item in pos:
    if item>=0:
        mask.append(1)
    else:
        mask.append(0)
print(mask)
xConv = x.to_numpy()
print(len(xConv[0]))
removals = []
for i in range(0,len(mask)-1):
    if mask[i] == 0:
        removals.append(i+1)

print(removals)
for removal in reversed(removals):
    xConv = np.delete(xConv,removal,1)
        
print(xConv)
# xFeatures = np.reshape(xConv,(-1,1))
xFeatures = pd.DataFrame(data=xConv)
print(xFeatures)
md = RandomForestClassifier(n_estimators = 100,\
                            max_depth=100, max_features="log2",\
                            min_samples_leaf=1, min_samples_split=4, n_jobs = -1)
md.fit(xFeatures,y)
prediction = md.predict(xFeatures)
accuracy = cross_val_score(md,X_train,y_train, cv=10).mean()
print(accuracy)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))
# [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
# 0.9780791538723985

[1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]
47
[2, 4, 6, 9, 10, 12, 13, 14, 15, 28, 29, 30, 40, 46]
[[0 3 0 ... 94694400 6 1]
 [0 7 0 ... 16502399 11 1]
 [0 8 0 ... -1 0 0]
 ...
 [0 10 0 ... 16502399 11 0]
 [0 11 0 ... -1 0 0]
 [0 9 0 ... 21470521 22 0]]
      0   1  2  3      4  5  6  7  8  9   ... 23 24 25 26         27  28   29  \
0      0   3  0  0  False  2  0  0  0  0  ...  0  0  0  0          0  24  412   
1      0   7  0  0  False  1  0  0  0  0  ...  0  0  0  0  0.0652174   1   25   
2      0   8  0  0  False  1  0  0  0  0  ...  0  0  0  0          0  18  969   
3      0   9  0  0  False  1  0  0  0  0  ...  0  0  0  0  0.0672269   1   25   
4      0   6  0  0  False  1  0  0  0  0  ...  0  0  0  0          0   1   25   
...   ..  .. .. ..    ... .. .. .. .. ..  ... .. .. .. ..        ...  ..  ...   
29808  0   8  0  0  False  1  0  0  0  0  ...  0  0  0  0          0   1   35   

In [ ]:
#BASE MODEL NN 1 layer
colLen = len(X_train.columns)
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dense(500))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
md.fit(X_train,y_train,50,20,1,validation_data = (X_test, y_test))
prediction = md.predict(X_test)

prediction = np.round(prediction)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))

prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#BASE MODEL NN 2 layer

colLen = len(X_train.columns)
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(500))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.5))

md.add(Dense(500))
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
md.fit(X_train,y_train,50,20,1,validation_data = (X_test, y_test))
prediction = md.predict(X_test)
prediction = np.round(prediction)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))
       
prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#BASE MODEL NN 3 layer

colLen = len(X_train.columns)
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(500))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.5))

md.add(Dense(500))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.5))

md.add(Dense(500))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
md.fit(X_train,y_train,50,20,1,validation_data = (X_test, y_test))
prediction = md.predict(X_test)
prediction = np.round(prediction)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))
       
prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
#basic tree based feature optimisation
estimator = RandomForestClassifier(n_estimators = 500, verbose=1, n_jobs = -1)
selector = RFE(estimator,round(len(x.columns)*0.6)) #perhaps a parameter to be optimised
selector = selector.fit(x,y)

In [ ]:
i = len(selector.support_)
for columnRequired in reversed(selector.support_):
    i-=1
    if(not columnRequired):
        x = x.drop(x.columns[i],axis=1)
x
    #if false delete

In [ ]:
#TPE optimisation 3 LAYER NN accuracy:


#remove all the unselected
space = {'neurons1':hp.uniform('neurons1',32,512),
         'neurons2':hp.uniform('neurons2',32,512),
         'neurons3':hp.uniform('neurons3',32,512),
         'dropout1':hp.uniform('dropout1',0.25,0.5),
         'dropout2':hp.uniform('dropout2',0.25,0.5),
         'dropout3':hp.uniform('dropout3',0.25,0.5),
         'batchSize':hp.uniform('batchSize',20,100),
         'optimiser':hp.choice('optimser',['RMSprop','SGD','Adadelta','Adam'])
        }
colLen = len(X_train.columns)
def nn(params):
    print("test of: ", params)
    model = Sequential()
    model.add(Dense(colLen,input_shape=(colLen,)))
    #needs to be low enough neurons to limit overfitting, but high enough to be useful
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(round(params['neurons1'])))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(params['dropout1']))
    model.add(Dense(round(params['neurons2'])))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(params['dropout2']))
    model.add(Dense(round(params['neurons3'])))
    model.add(Activation('relu'))
    model.add(Dropout(params['dropout3']))
    model.add(Dense(1, activation='sigmoid'))
    #normalises data into a probability distribution
    
    model.compile(params['optimiser'],loss=binary_crossentropy, metrics=["accuracy"])
    model.fit(X_train,y_train,round(params['batchSize']),20,0,validation_data = (X_test, y_test))
    predAcc = model.predict_proba(X_test,batch_size=128,verbose=0)
    acc = roc_auc_score(y_test,predAcc)
    import sys
    sys.stdout.flush() 
    return{'loss':-acc,'status':STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(nn, space, algo=tpe.suggest, max_evals=100, trials=trials)
print('best:', best)

In [ ]:
colLen = len(X_train.columns)
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(371))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.25))

md.add(Dense(395))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.41))

md.add(Dense(52))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.31))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
md.fit(X_train,y_train,68,20,1,validation_data = (X_test, y_test))
prediction = md.predict(X_test)
prediction = np.round(prediction)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))
       
prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
from deephyper.search.nas.model.space import KSearchSpace
def nn(params):
    colLen = len(X_train.columns)
    struct = KSearchSpace(colLen)
    print("test of: ", params)
    model = Sequential()
    model.add(Dense(colLen,input_shape=(colLen,)))
    #needs to be low enough neurons to limit overfitting, but high enough to be useful
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(round(params['neurons1'])))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(params['dropout1']))
    model.add(Dense(round(params['neurons2'])))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(params['dropout2']))
    model.add(Dense(round(params['neurons3'])))
    model.add(Activation('relu'))
    model.add(Dropout(params['dropout3']))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(params['optimiser'],loss=binary_crossentropy, metrics=["accuracy"])
    model.fit(X_train,y_train,round(params['batchSize']),20,0,validation_data = (X_test, y_test))
    predAcc = model.predict_proba(X_test,batch_size=128,verbose=0)
    acc = roc_auc_score(y_test,predAcc)
    import sys
    sys.stdout.flush() 
    return{'loss':-acc,'status':STATUS_OK}

In [ ]:
#I feel there is no cross validaiton in the algorithm and so the model is simply overfitting
#As the accuracy does nto carry over to a seperate test.
space = {
    "epochs":[20],
    "batch_size":[20,30,40,50],
    "n_layers":[2,3],
    "n_neurons":[100,200,300,400],
    "dropout":[0.1,0.25,0.5],
    "optimizers":['RMSprop','SGD','Adadelta','Adam'],
    "activations":["relu"],
    "last_layer_activations":["sigmoid"],
    "losses":["binary_crossentropy"],
    "metrics":["accuracy"]
}
from neuro_evolution import evolution
search = evolution.NeuroEvolution(generations = 15, population = 100, params = space)
search.evolve(X_train,y_train,X_test,y_test)

In [ ]:


colLen = len(X_train.columns)
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(300))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.1))

md.add(Dense(300))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.1))

md.add(Dense(300))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.1))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
md.fit(X_train,y_train,50,20,1,validation_data = (X_test, y_test))
prediction = md.predict(X_test)
prediction = np.round(prediction)
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test, prediction))
       
prediction = md.predict(x1)
print(df)
df["prediction"] = prediction
print(prediction)
df.to_csv("results.csv")

In [ ]:
options = {'c1':0.5,'c2':0.5,'w':0.2,'k':15,'p':2}
dimensions = len(x.columns)
optimiser = pso.discrete.BinaryPSO(n_particles=15, dimensions=dimensions, options=options)

cost, pos = optimiser.optimize(loopPso,iters = 5)

In [ ]:
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
colLen = xFeatures.shape[1]
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(371))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.25))

md.add(Dense(395))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.41))

md.add(Dense(52))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.31))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
md.fit(xFeatures,y,68,10)
prediction = md.predict(xFeatures)
prediction = np.round(prediction)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [ ]:
colLen = 47
md = Sequential()
md.add(Dense(colLen,input_shape=(colLen,)))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(371))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.25))

md.add(Dense(395))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.41))

md.add(Dense(52))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.31))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
import GAFO
test = GAFO.makeGene(0.01,15,30)
test.fit(md,"classification",x,y,False)
print(test.results())

In [ ]:
pos = test.results()[0]
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
md = Sequential()
md.add(Dense(27))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(371))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.25))

md.add(Dense(395))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.41))

md.add(Dense(52))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.31))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
print(y)
y = y.to_numpy()
md.fit(xFeatures,y,68,10)
prediction = md.predict(xFeatures)
prediction = np.round(prediction)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [ ]:
md =BaseMFO(MFOObjectiveFuncNN,47,(0,1),epochT=10, pop_size=15)
pos, fit, loss = md._train__()
print(fit)    

In [ ]:
print(pos)    
mask = np.reshape(pos/2,(1,-1))
mask = np.round(mask)
mask = mask[0]
print(mask)

In [ ]:
mask = np.reshape(pos,(1,-1))
mask = np.round(mask)
mask = mask[0]
xConv = x.to_numpy()
xFeatures = xConv[:,mask==1]
md = Sequential()
md.add(Dense(27))
#needs to be low enough neurons to limit overfitting, but high enough to be useful
md.add(BatchNormalization())
md.add(Activation('relu'))

md.add(Dense(371))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.25))

md.add(Dense(395))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.41))

md.add(Dense(52))
md.add(BatchNormalization())
md.add(Activation('relu'))
md.add(Dropout(0.31))
md.add(Dense(1, activation='sigmoid'))
#normalises data into a probability distribution
md.compile('Adam',loss=binary_crossentropy, metrics=["accuracy"])
print(y)
y = y.to_numpy()
md.fit(xFeatures,y,68,10)
prediction = md.predict(xFeatures)
prediction = np.round(prediction)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [84]:
with open('detectionModel2.pkl','rb') as md:
    model = pickle.load(md)

In [14]:
print(x.columns)

Index(['dash', 'domainLength', 'underscore', 'at', 'slash', 'equals', 'http',
       'www', 'dot', 'digit', 'coukTld', 'govTld', 'eduTld', 'netTld', 'ruTld',
       'containsLog', 'containsPay', 'containsWeb', 'containsCmd',
       'containsAccount', 'containsDispatch', 'containsHttps', 'orgTld',
       'jpTld', 'comBrTld', 'deTld', 'cnTld', 'irTld', 'tvTld', 'coJpTld',
       'grTld', 'plTld', 'esTld', 'infoTld', 'appTld', 'comAuTld', 'mxTld',
       'comTrTld', 'ipInUrl', 'linksToNowhere', 'linksToExternal',
       'urlCountry', 'asn', 'certificateDuration', 'certificateIssuer',
       'googleFrontPage', 'googleSafeBrowsing'],
      dtype='object')


In [21]:
pos =np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
print(xFeatures)
prediction = model.predict(xFeatures)
prediction = np.round(prediction)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))
       

[[0 11 0 ... 1 0 0]
 [0 7 0 ... 8 1 0]
 [0 5 0 ... 19 1 0]
 ...
 [0 11 0 ... 8 0 0]
 [0 12 0 ... 1 1 0]
 [0 12 0 ... 2 0 0]]
[[ 9526   626]
 [  809 12487]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     10152
           1       0.95      0.94      0.95     13296

    accuracy                           0.94     23448
   macro avg       0.94      0.94      0.94     23448
weighted avg       0.94      0.94      0.94     23448

0.9388007505970658


In [ ]:
pos =np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1])
xConv = x.to_numpy()
xFeatures = xConv[:,pos==1]
print(xFeatures)
prediction = model.predict(xFeatures)
prediction = np.round(prediction)
print(confusion_matrix(y,prediction))
print(classification_report(y,prediction))
print(accuracy_score(y, prediction))

In [ ]:
#Anything below was used for statistical analysis

con = sqlite3.connect(DATABASE_DIRECTORY)
# cur = con.cursor()
# cur.execute("SELECT url, urlClassification FROM Url")
# results = cur.fetchall()
# #urlId, url, asn, classification, country, hash, ip, loadingTime,size,status,uuid
# df = pd.DataFrame(data = results, columns=["url","classification"])
# tldArray = []
# for result in results:
#     url = result[0]
#     brokenUrl = tldextract.extract(url)
#     tld = brokenUrl.suffix
# #     print(tld)
#     tldArray.append(tld)
# assert len(df) == len(tldArray)
# df["tld"] = tldArray
# dfBl = df[df.classification == 1]
# dfWl = df[df.classification == 0]
# #using the below will find the count of a specific TLD for whitelist or blacklist
# #if both are commented out, the top and bottom 25 counts are shown instead.
# # dfWl = dfWl[df.tld == "com.tr"]
# dfBl = dfBl[df.tld == "co.kr"]
# """
# I prefer the output that jupyter shows, if you wish to see both you can add
# a print statement to both of these instead, otherwise it'll show only the list
# thats uncommented, or the last one if both are uncommented.
# """
# # dfWl.groupby('tld').nunique().sort_values('url', ascending=False)
# dfBl.groupby('tld').nunique().sort_values('url',ascending=False)


In [ ]:
# importances = list(md.feature_importances_)
# feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(list(X_train.columns.values), importances)]# Sort the feature importances by most important first
# feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
# [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# from sklearn.tree import export_graphviz
# import os
# import pydot
# estimator = md.estimators_[9]
# print(list(dfPred.columns.values))
# export_graphviz(estimator, 
#                 out_file = 'small_tree.dot', 
#                 feature_names = list(X_train.columns.values),
#                 rounded = True, precision = 1)
# (graph, ) = pydot.graph_from_dot_file('small_tree.dot')
# graph.write_png('small_tree.png')

# (graph, ) = pydot.graph_from_dot_file('small_tree.dot')
# graph.write_png('small_tree.png');
# #export_graphviz(estimator, feature_names = list(X_train.columns.values),
# #                 filled=True,
# #                 rounded=True)
# # from subprocess import call
# # call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# # # Display in jupyter notebook
# # from IPython.display import Image
# # Image(filename = 'tree.png')

In [ ]:
# testdf = pd.read_csv("urlset.csv")
# print(testdf["label"])
# testdf = testdf[["domain","label"]]
# for domain in testdf:
#     url = result[0]
#     brokenUrl = tldextract.extract(url)
#     tld = brokenUrl.suffix
# #     print(tld)
#     tldArray.append(tld)
# assert len(df) == len(tldArray)
# df["tld"] = tldArray
# dfBl = df[df.classification == 1]
# dfWl = df[df.classification == 0]
# dfWl = dfWl[df.tld == "com.tr"]
# dfWl.groupby('tld').nunique().sort_values('url', ascending=False)
# brokenUrl = tldextract.extract(url)
# tld = brokenUrl.suffix

